# reference fasta libraries:

In [4]:
%%bash
cd ~/Downloads/base
mkdir -p test_smallonly
cd ~/Downloads/base/test_smallonly
rm -rf /home/guillaume/Downloads/base/test_smallonly/*

# references:
hairpin='/home/guillaume/Homo_sapiens_UCSC_hg38/hairpin_hsa.fa'
rRNA='/home/guillaume/Downloads/base/israp/rRNA.fa'
mature_hsa_filtered_T="/home/guillaume/Homo_sapiens_UCSC_hg38/mature_hsa_filtered_T.fa"
smallRNA="/home/guillaume/Homo_sapiens_UCSC_hg38/smallRNA.fa"
grep ">" $rRNA | wc -l
grep ">" $mature_hsa_filtered_T | wc -l
grep ">" $smallRNA | wc -l

# choose reference:
#cat $hairpin $rRNA $mature_hsa_filtered_T $smallRNA > combined.fasta
cat $mature_hsa_filtered_T > combined.fasta
cp combined.fasta reference.fasta

# if necessary, replace U with T in .fa file
sed -i s/U/T/g ./reference.fasta

# if necessary, replace lower case atgc with uppercase ATGC in .fa file
awk '{ if ($0 !~ />/) {print toupper($0)} else {print $0} }' reference.fasta > reference2.fasta

# if necessary, replace Y with N 
perl -e 'while(<>) { if ($_ =~ /^>.*/) { print $_; } else { $_ =~ tr/Y/N/; print $_;}}' < reference2.fasta > reference.fasta

/home/guillaume/.local/bin/fasta_formatter -i reference2.fasta -o reference.oneline.fasta -w 0

565
2588
672177


In [6]:
# collapse duplicate sequences in reference:
#/home/guillaume/.local/bin/fastx_collapser < reference.oneline.fasta > reference.collapsed.fasta
inname='/home/guillaume/Downloads/base/test_smallonly/reference.oneline.fasta'
oname='/home/guillaume/Downloads/base/test_smallonly/reference.collapsed.fasta'

from itertools import groupby

if __name__ == '__main__':

    ishead = lambda x: x.startswith('>')
    all_seqs = set()
    with open(inname) as handle:
        with open(oname, 'w') as outhandle:
            head = None
            for h, lines in groupby(handle, ishead):
                if h:
                    head = lines.__next__()
                else:
                    seq = ''.join(lines)
                    if seq not in all_seqs:
                        all_seqs.add(seq)
                        outhandle.write('%s%s' % (head, seq))

In [7]:
%%bash
cd ~/Downloads/base/test_smallonly
cp reference.collapsed.fasta reference.fasta 
/home/guillaume/.local/bin/fastx_quality_stats -i reference.fasta -o reference.fasta.quality.txt
/home/guillaume/.local/bin/fastx_nucleotide_distribution_graph.sh -i reference.fasta.quality.txt -o reference.distrib.png

# report info
grep ">" reference.fasta | wc -l

2565


In [8]:
%%bash
cd ~/Downloads/base/test_smallonly
/home/guillaume/.local/bin/fastx_quality_stats -i reference.fasta
# if necessary, index reference:
~/Downloads/bwa.kit/bwa index reference.fasta
##bowtie2-build reference.fasta reference

column	count	min	max	sum	mean	Q1	med	Q3	IQR	lW	rW	A_Count	C_Count	G_Count	T_Count	N_Count	Max_count
1	2565	100	-100	0	0.00	52	260	367	315	100	-100	719	556	358	932	0	2565
2	2565	100	-100	0	0.00	52	260	262	210	100	-100	684	570	743	568	0	2565
3	2565	100	-100	0	0.00	52	260	262	210	100	-100	665	542	790	568	0	2565
4	2565	100	-100	0	0.00	155	262	262	107	100	-100	544	603	939	479	0	2565
5	2565	100	-100	0	0.00	155	262	262	107	100	-100	562	547	898	558	0	2565
6	2565	100	-100	0	0.00	155	260	262	107	100	-100	574	657	738	596	0	2565
7	2565	100	-100	0	0.00	52	260	262	210	100	-100	643	619	713	590	0	2565
8	2565	100	-100	0	0.00	155	260	365	210	100	-100	575	649	689	652	0	2565
9	2565	100	-100	0	0.00	157	262	365	208	100	-100	533	621	691	720	0	2565
10	2565	100	-100	0	0.00	157	262	365	208	100	-100	519	638	756	652	0	2565
11	2565	100	-100	0	0.00	155	262	262	107	100	-100	549	602	820	594	0	2565
12	2565	100	-100	0	0.00	155	262	365	210	100	-100	589	580	717	679	0	2565
13	2565	100	-100	0	0.00	155	262	365	210	100	-100	

[bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.02 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.01 sec
[bwa_index] Construct SA from BWT and Occ... 0.01 sec
[main] Version: 0.7.12-r1039
[main] CMD: /home/guillaume/Downloads/bwa.kit/bwa index reference.fasta
[main] Real time: 0.465 sec; CPU: 0.040 sec


choose sample, remove adapters and select read length range:

# merge PEAR

In [9]:
%%bash
cd ~/Downloads/base
mkdir -p test_smallonly
mkdir -p test_smallonly/peared
rm -rf ./test_smallonly/peared/*

# select sample:
samples="RF-118 RF-115 RF-117 RF-59 RF-66 RF67 RF55-1 RF112 RF111 RF105 RF104 RF79 RF1 106-1 109-2 110-3 113 107 108-1 RF-102 RF-85D RF-81 RF-80 RF-62 RF-50 RF-49 RF-42 RF-32 RF-31 RF-30 RF-20 RF-17 RF-16 RF-12 RF-10 RF-5 RF-4"

# remove adapter and trim reads:
for sample in $samples; do
    echo $sample
    fileR1=$(ls | grep .*${sample}_.*R1.*)
    fileR2=$(ls | grep .*${sample}_.*R2.*)
    echo $fileR1
    echo $fileR2

~/.local/bin/cutadapt -f fastq -e 0.1 -q 20,20 -O 1 -m 16 -M 200 --max-n 2 --discard-untrimmed --match-read-wildcards \
-a AGATCGGAAGAGCACACGTCT -A GATCGTCGGACTGTAGAACTCTGAAC \
-o ./cutadapt_paired/$fileR1 -p ./cutadapt_paired/$fileR2 $fileR1 $fileR2

pear -f ./cutadapt_paired/$fileR1 -r ./cutadapt_paired/$fileR2 -o ./test_smallonly/peared/$fileR1 -j 4 -u 1 -y 10G -n 0
    rm ./test_smallonly/peared/*unassembled*
    rm ./test_smallonly/peared/*discarded*
    done


Process is interrupted.


# align

In [10]:
%%bash
cd ~/Downloads/base/test_smallonly

# select sample:
samples="RF-118 RF-115 RF-117 RF-59 RF-66 RF67 RF55-1 RF112 RF111 RF105 RF104 RF79 RF1 106-1 109-2 110-3 113 107 108-1 RF-102 RF-85D RF-81 RF-80 RF-62 RF-50 RF-49 RF-42 RF-32 RF-31 RF-30 RF-20 RF-17 RF-16 RF-12 RF-10 RF-5 RF-4"
#samples="RF-118 RF-115"

mkdir -p bams
rm -rf ./bams/*
mkdir -p counts
rm -rf ./counts/*
mkdir -p stat_mapped
rm -rf ./stat_mapped/*

for sample in $samples; do
    fileR1=$(ls ./peared/ | grep ${sample}_.*R1.*)
    fileR2=$(ls ./peared/ | grep ${sample}_.*R2.*)
    
    ~/Downloads/bwa.kit/bwa aln -n 1 -o 0 -e 0 -k 1 -t 4 reference.fasta ./peared/$fileR1 > R1.sai 2> error
    ~/Downloads/bwa.kit/bwa samse reference.fasta R1.sai ./peared/$fileR1 > $fileR1.sam 2> error
    # sam to bam 
    samtools view -Sb $fileR1.sam > view.bam
    mv view.bam $fileR1.bam
    # bam to sorted bam
    samtools sort $fileR1.bam -o $fileR1.sorted.bam
    # sorted bam to index sorted bam
    samtools index $fileR1.sorted.bam
    # convert to bed
    bamToBed -i $fileR1.sorted.bam > $fileR1.sorted.bed
    # generate statistics: percentage of patients reads mapped to genome hg38
    samtools flagstat $fileR1.sorted.bam > $fileR1.percentage_mapped.txt
    samtools idxstats $fileR1.sorted.bam > $fileR1.count.txt
      
    ### now trim 1 base of unmapped reads
    samtools view -b -f 4 $fileR1.sorted.bam > unmapped.bam
    samtools sort -n -O BAM -o aln.qsort.bam unmapped.bam 
    bedtools bamtofastq -i aln.qsort.bam -fq unmapped.fq
    cat unmapped.fq | paste - - - - | cut -f1-2 | sed 's/^@/>/g' | tr '\t' '\n' > unmapped.fasta
    revseq unmapped.fasta -reverse -nocomplement -outseq  unmapped.rev.fasta
    /home/guillaume/.local/bin/fasta_formatter -i unmapped.rev.fasta -o unmapped.rev.oneline.fasta -w 0
    /home/guillaume/.local/bin/fastx_trimmer -f 2 -i unmapped.rev.oneline.fasta -o unmapped.rev.trimmed1.fasta
    revseq unmapped.rev.trimmed1.fasta -reverse -nocomplement -outseq  unmapped.trimmed1.fasta
    /home/guillaume/.local/bin/fasta_formatter -i unmapped.trimmed1.fasta -o unmapped.trimmed1.ol.fasta -w 0  
    sed -n '1~2p' unmapped.fasta > firstline
    sed -n '0~2p' unmapped.trimmed1.ol.fasta > secondline
    paste -d"\n" firstline secondline > $fileR1.unmapped.trimmed1.fasta
    
    ## align trimmed1
    ~/Downloads/bwa.kit/bwa aln -n 1 -o 0 -e 0 -k 1 -t 4 reference.fasta $fileR1.unmapped.trimmed1.fasta > R1.sai 2> error
    ~/Downloads/bwa.kit/bwa samse reference.fasta R1.sai $fileR1.unmapped.trimmed1.fasta > $fileR1.t1.sam 2> error
    # sam to bam 
    samtools view -Sb $fileR1.t1.sam > view.bam
    mv view.bam $fileR1.t1.bam
    # bam to sorted bam
    samtools sort $fileR1.t1.bam -o $fileR1.sorted.t1.bam
    # sorted bam to index sorted bam
    samtools index $fileR1.sorted.t1.bam
    # convert to bed
    bamToBed -i $fileR1.sorted.t1.bam > $fileR1.sorted.t1.bed
    # generate statistics: percentage of patients reads mapped to genome hg38
    samtools flagstat $fileR1.sorted.t1.bam > $fileR1.percentage_mapped.t1.txt
    samtools idxstats $fileR1.sorted.t1.bam > $fileR1.count.t1.txt   
    
        ### now trim 1 base of unmapped reads
    samtools view -b -f 4 $fileR1.sorted.t1.bam > unmapped.bam
    samtools sort -n -O BAM -o aln.qsort.bam unmapped.bam 
    bedtools bamtofastq -i aln.qsort.bam -fq unmapped.fq
    cat unmapped.fq | paste - - - - | cut -f1-2 | sed 's/^@/>/g' | tr '\t' '\n' > unmapped.fasta
    revseq unmapped.fasta -reverse -nocomplement -outseq  unmapped.rev.fasta
    /home/guillaume/.local/bin/fasta_formatter -i unmapped.rev.fasta -o unmapped.rev.oneline.fasta -w 0
    /home/guillaume/.local/bin/fastx_trimmer -f 2 -i unmapped.rev.oneline.fasta -o unmapped.rev.trimmed1.fasta
    revseq unmapped.rev.trimmed1.fasta -reverse -nocomplement -outseq  unmapped.trimmed1.fasta
    /home/guillaume/.local/bin/fasta_formatter -i unmapped.trimmed1.fasta -o unmapped.trimmed1.ol.fasta -w 0  
    sed -n '1~2p' unmapped.fasta > firstline
    sed -n '0~2p' unmapped.trimmed1.ol.fasta > secondline
    paste -d"\n" firstline secondline > $fileR1.unmapped.trimmed2.fasta
    
    ## align trimmed2
    ~/Downloads/bwa.kit/bwa aln -n 1 -o 0 -e 0 -k 1 -t 4 reference.fasta $fileR1.unmapped.trimmed2.fasta > R1.sai 2> error
    ~/Downloads/bwa.kit/bwa samse reference.fasta R1.sai $fileR1.unmapped.trimmed2.fasta > $fileR1.t2.sam 2> error
    # sam to bam 
    samtools view -Sb $fileR1.t2.sam > view.bam
    mv view.bam $fileR1.t2.bam
    # bam to sorted bam
    samtools sort $fileR1.t2.bam -o $fileR1.sorted.t2.bam
    # sorted bam to index sorted bam
    samtools index $fileR1.sorted.t2.bam
    # convert to bed
    bamToBed -i $fileR1.sorted.t2.bam > $fileR1.sorted.t2.bed
    # generate statistics: percentage of patients reads mapped to genome hg38
    samtools flagstat $fileR1.sorted.t2.bam > $fileR1.percentage_mapped.t2.txt
    samtools idxstats $fileR1.sorted.t2.bam > $fileR1.count.t2.txt   


    ### now trim 1 base of unmapped reads
    samtools view -b -f 4 $fileR1.sorted.t2.bam > unmapped.bam
    samtools sort -n -O BAM -o aln.qsort.bam unmapped.bam 
    bedtools bamtofastq -i aln.qsort.bam -fq unmapped.fq
    cat unmapped.fq | paste - - - - | cut -f1-2 | sed 's/^@/>/g' | tr '\t' '\n' > unmapped.fasta
    revseq unmapped.fasta -reverse -nocomplement -outseq  unmapped.rev.fasta
    /home/guillaume/.local/bin/fasta_formatter -i unmapped.rev.fasta -o unmapped.rev.oneline.fasta -w 0
    /home/guillaume/.local/bin/fastx_trimmer -f 2 -i unmapped.rev.oneline.fasta -o unmapped.rev.trimmed1.fasta
    revseq unmapped.rev.trimmed1.fasta -reverse -nocomplement -outseq  unmapped.trimmed1.fasta
    /home/guillaume/.local/bin/fasta_formatter -i unmapped.trimmed1.fasta -o unmapped.trimmed1.ol.fasta -w 0  
    sed -n '1~2p' unmapped.fasta > firstline
    sed -n '0~2p' unmapped.trimmed1.ol.fasta > secondline
    paste -d"\n" firstline secondline > $fileR1.unmapped.trimmed3.fasta
    
    ## align trimmed3
    ~/Downloads/bwa.kit/bwa aln -n 1 -o 0 -e 0 -k 1 -t 4 reference.fasta $fileR1.unmapped.trimmed3.fasta > R1.sai 2> error
    ~/Downloads/bwa.kit/bwa samse reference.fasta R1.sai $fileR1.unmapped.trimmed3.fasta > $fileR1.t3.sam 2> error
    # sam to bam 
    samtools view -Sb $fileR1.t3.sam > view.bam
    mv view.bam $fileR1.t3.bam
    # bam to sorted bam
    samtools sort $fileR1.t3.bam -o $fileR1.sorted.t3.bam
    # sorted bam to index sorted bam
    samtools index $fileR1.sorted.t3.bam
    # convert to bed
    bamToBed -i $fileR1.sorted.t3.bam > $fileR1.sorted.t3.bed
    # generate statistics: percentage of patients reads mapped to genome hg38
    samtools flagstat $fileR1.sorted.t3.bam > $fileR1.percentage_mapped.t3.txt
    samtools idxstats $fileR1.sorted.t3.bam > $fileR1.count.t3.txt   

    ## merge the 4 bam files per sample
    samtools merge -f $fileR1.full.bam $fileR1.sorted.bam $fileR1.sorted.t1.bam $fileR1.sorted.t2.bam $fileR1.sorted.t3.bam
    samtools sort $fileR1.full.bam -o $fileR1.full.sorted.bam
    samtools index $fileR1.full.sorted.bam
    mv $fileR1.full.sorted.bam.bai ./bams/ 
    mv $fileR1.full.sorted.bam ./bams/
    samtools idxstats  ./bams/$fileR1.full.sorted.bam > ./counts/$fileR1.full.count.txt
    mv *percentage* ./stat_mapped/
    
    done

Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
[bam_sort_core] merging from 3 files...
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
[bam_sort_core] merging from 2 files...
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
Reverse and complement a nucleotide sequence
[bam_sort_core] merging from 3 files...
Reverse and complement a nucleotide sequence
Reverse and complement a 